# SmartProjectionPoseFactor

`SmartProjectionPoseFactor<CALIBRATION>` is a "smart" factor optimized for visual SLAM problems where the **camera calibration is known and fixed**, and only the camera **poses** are being optimized.
It inherits from `SmartProjectionFactor` but simplifies the setup by taking a single shared `CALIBRATION` object (e.g., `Cal3_S2`, `Cal3Bundler`) assumed to be used by all cameras observing the landmark.

Key characteristics:
- **Implicit Point:** Like its base class, the 3D point is not an explicit variable.
- **Fixed Calibration:** Assumes a single, known calibration `K` for all measurements.
- **Pose Variables:** The factor connects `Pose3` variables directly.
- **`Values` Requirement:** Requires `Pose3` objects in the `Values` container for its keys.
- **Configuration:** Behavior is controlled by `SmartProjectionParams`.

**Use Case:** Ideal for visual SLAM or SfM when camera intrinsics are pre-calibrated and assumed constant.
**Alternative:** If calibration also needs optimization, use `SmartProjectionFactor`.

If you are using the factor, please cite:
> **L. Carlone, Z. Kira, C. Beall, V. Indelman, F. Dellaert**, "Eliminating conditionally independent sets in factor graphs: a unifying perspective based on smart factors", Int. Conf. on Robotics and Automation (ICRA), 2014.

<a href="https://colab.research.google.com/github/borglab/gtsam/blob/develop/gtsam/slam/doc/SmartProjectionPoseFactor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    import google.colab
    %pip install --quiet gtsam-develop
except ImportError:
    pass  # Not running on Colab, do nothing

In [2]:
import gtsam
from gtsam import (
    Values,
    Point2,
    Point3,
    Pose3,
    Rot3,
    NonlinearFactorGraph,
    SmartProjectionParams,
    SmartProjectionPoseFactorCal3_S2,
    Cal3_S2,
)
from gtsam.symbol_shorthand import X  # Key for Pose3 variable

## Creating and Adding Measurements

1. Create the factor with a noise model, the shared calibration `K`, and parameters.
2. Add measurements (2D points) and the corresponding pose keys.

In [3]:
smart_noise = gtsam.noiseModel.Isotropic.Sigma(2, 1.0)
smart_params = SmartProjectionParams() # Use default params
K = Cal3_S2(500, 500, 0, 320, 240) # Fixed calibration

smart_factor = SmartProjectionPoseFactorCal3_S2(smart_noise, K, smart_params)

# Add measurements and keys (Pose keys)
smart_factor.add(Point2(150, 505), X(0))
smart_factor.add(Point2(470, 495), X(1))
smart_factor.add(Point2(480, 150), X(2))

print(f"Smart factor involves {smart_factor.size()} measurements.")
smart_factor.print("SmartFactorPose: ")

Smart factor involves 3 measurements.
SmartFactorPose: SmartProjectionPoseFactor, z = 
 SmartProjectionFactor
linearizationMode: 0
triangulationParameters:
rankTolerance = 1
enableEPI = 0
landmarkDistanceThreshold = -1
dynamicOutlierRejectionThreshold = -1
useLOST = 0
noise model

result:
no point, status = 1

SmartFactorBase, z = 
measurement 0, px = 
150
505
noise model = unit (2) 
measurement 1, px = 
470
495
noise model = unit (2) 
measurement 2, px = 
480
150
noise model = unit (2) 
  keys = { x0 x1 x2 }


## Evaluating the Error

The `.error(values)` method uses the `Pose3` objects from `values` and the fixed `K` to triangulate the point and compute the error.

In [4]:
# Create Values containing Pose3 objects
values = Values()
pose0 = Pose3(Rot3.Ypr(0.1, 0, 0), Point3(-1, 0, 0.5))
pose1 = Pose3(Rot3.Ypr(0.0, -0.1, 0), Point3(1, 0, 0.5))
pose2 = Pose3(Rot3.Ypr(0, 0.0, 0.2), Point3(0, 1, 0.5))

values.insert(X(0), pose0)
values.insert(X(1), pose1)
values.insert(X(2), pose2)

# Triangulate first to see the implicit point
point_result = smart_factor.point(values)
print(f"Triangulated point result: {point_result.status}")

if point_result.valid():
   print(f"Triangulated point: {point_result.get()}")
   # Calculate error
   total_error = smart_factor.error(values)
   print(f"\nTotal reprojection error (0.5 * sum(err^2/sigma^2)): {total_error:.4f}")
else:
   print("\nTriangulation failed, error calculation depends on degeneracyMode.")

Triangulated point result: Status.VALID
Triangulated point: [0.28416823 1.95555615 5.67688675]

Total reprojection error (0.5 * sum(err^2/sigma^2)): 120243.1626


## Linearization

Linearization works similarly to `SmartProjectionFactor`, producing a linear factor (default: Hessian) connecting the `Pose3` variables.

In [5]:
graph = NonlinearFactorGraph()
graph.add(smart_factor)

# Linearize (default mode is HESSIAN)
linear_factor = smart_factor.linearize(values)

if linear_factor:
    print("\nLinearized Factor (showing HessianFactor structure):")
    # Cast to HessianFactor to print its details
    hessian_factor = gtsam.HessianFactor(linear_factor)
    if hessian_factor:
         hessian_factor.print()
    else:
         print("Linearized factor is not a HessianFactor (check params.linearizationMode)")
else:
    print("Linearization failed (likely due to triangulation degeneracy)")


Linearized Factor (showing HessianFactor structure):

 keys: x0(6) x1(6) x2(6) 
Augmented information matrix: [
	202102, 6373.47, -59741.9, 4386.91, -35145.2, 11091.1, -115337, -44197.1, 12233.6, -8181.39, 19268.4, -7521.18, -81683.2, -724.877, 5044.41, -484.005, 14404.6, -5767.48, 89097.4;
	6373.47, 81115.2, -30292.7, 14717.5, -2404.93, -3343.21, 63410.4, 20584.6, -5335.01, 3787.66, -10602.2, 4111.71, -60743.7, -88128.4, 38992.5, -18242.3, 11072.3, -3393.47, 105458;
	-59741.9, -30292.7, 27634, -6415.45, 10844.3, -1981.98, 10553.8, 5348.27, -1607.72, 998.024, -1760.06, 696.401, 44568.2, 31147.8, -15125.4, 6542.51, -7986, 2832.46, -62376.9;
	4386.91, 14717.5, -6415.45, 2722.09, -996.496, -424.653, 9577.1, 3000.7, -765.229, 551.36, -1601.54, 620.327, -12253.7, -15890.6, 7106.5, -3294.65, 2225.84, -703.855, 20429.2;
	-35145.2, -2404.93, 10844.3, -996.496, 6132.48, -1869.56, 18982.5, 7333.62, -2035.73, 1357.9, -3171.11, 1238.23, 15136.7, 1537.88, -1499.49, 376.242, -2675.09, 1054.42, -171